# Quickstart for creating a Vertex Vector Search Index

In [1]:
!pwd

/home/jupyter/vector-io/notebooks


## Setup

In [2]:
# !gcloud services enable compute.googleapis.com \
#                         aiplatform.googleapis.com \
#                         storage.googleapis.com \
#                         iam.googleapis.com

### pip installs

In [3]:
# !pip install --upgrade --user google-cloud-aiplatform google-cloud-storage

In [4]:
# # Restart kernel after installs so that your environment can access the new packages
# import IPython
# import time

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### imports

In [5]:
import os 
import math
import time
import json
import uuid
import functools
import numpy as np
import pandas as pd
from pprint import pprint
from datetime import datetime

from google.cloud import aiplatform
from google.cloud import aiplatform_v1 as aipv1
from google.cloud import storage
from google.cloud import bigquery

# logging
import logging
logging.disable(logging.WARNING)

#python warning 
import warnings
warnings.filterwarnings("ignore")

In [6]:
from tqdm.auto import tqdm
import pyarrow.parquet as pq
from pyarrow import json as pj
from concurrent.futures import ThreadPoolExecutor
from typing import Generator, List, Tuple, Dict, Any, Optional

from vertexai.preview.language_models import TextEmbeddingModel
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2024-01-30 18:34:06.922780: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
print(f'BigQuery SDK version      : {bigquery.__version__}')
print(f'Vertex AI SDK version     : {aiplatform.__version__}')
print(f'Cloud Storage SDK version : {storage.__version__}')

BigQuery SDK version      : 3.15.0
Vertex AI SDK version     : 1.39.0
Cloud Storage SDK version : 2.14.0


## Set vars and GCP config

`CREATE_NEW_ASSETS`
* True creates new GCS buckets and Vector Search instances, etc.
* False skips these steps (in case you need to re-run notebook to include new variables you create)

In [9]:
# create new gcs bucket, vs index, etc.?
CREATE_NEW_ASSETS = False 

**Edit these to define naming conventions and stay organized across different versions**

In [10]:
# naming convention for all cloud resources
VERSION        = "pubv3"                     # TODO
PREFIX         = f'vvs-vectorio-{VERSION}'   # TODO

print(f"PREFIX = {PREFIX}")

PREFIX = vvs-vectorio-pubv3


### Edit these

In [11]:
# locations / regions for cloud resources
REGION            = 'us-central1'        
BQ_REGION         = 'US'

print(f"REGION    = {REGION}")
print(f"BQ_REGION = {BQ_REGION}")

REGION    = us-central1
BQ_REGION = US


### Let these populate

In [12]:
# let these ride
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

PROJECT_NUM              = !gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NUM              = PROJECT_NUM[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

# service account
VERTEX_SA                = f'{PROJECT_NUM}-compute@developer.gserviceaccount.com'

print(f"PROJECT_ID       = {PROJECT_ID}")
print(f"PROJECT_NUM      = {PROJECT_NUM}")
print(f"BUCKET_NAME      = {BUCKET_NAME}")
print(f"BUCKET_URI       = {BUCKET_URI}")
print(f"VERTEX_SA        = {VERTEX_SA}")

PROJECT_ID       = hybrid-vertex
PROJECT_NUM      = 934903580331
BUCKET_NAME      = vvs-vectorio-pubv3-hybrid-vertex
BUCKET_URI       = gs://vvs-vectorio-pubv3-hybrid-vertex
VERTEX_SA        = 934903580331-compute@developer.gserviceaccount.com


In [13]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


In [14]:
if CREATE_NEW_ASSETS:
    
    # create new bucket
    ! gsutil mb -l $REGION $BUCKET_URI
    
    # ### give Service account Admin to GCS
    # !gcloud projects add-iam-policy-binding $PROJECT_ID \
    #     --member=serviceAccount:$VERTEX_SA \
    #     --role=roles/storage.admin
    
    ### uncomment if org policy prevents granting Admin:
    # ! gsutil iam ch serviceAccount:{$VERTEX_SA}:roles/storage.objects.get $BUCKET_URI
    # ! gsutil iam ch serviceAccount:{$VERTEX_SA}:roles/storage.objects.create $BUCKET_URI
    # ! gsutil iam ch serviceAccount:{$VERTEX_SA}:roles/storage.objects.list $BUCKET_URI
    
    
print(f"{VERTEX_SA} should have access to {BUCKET_URI}")

934903580331-compute@developer.gserviceaccount.com should have access to gs://vvs-vectorio-pubv3-hybrid-vertex


### Initialize Google Cloud SDK Clients

In [15]:
# cloud storage client
storage_client = storage.Client(project=PROJECT_ID)

# Vertex client
aiplatform.init(project=PROJECT_ID, location=REGION)

# bigquery client
bq_client = bigquery.Client(
    project=PROJECT_ID,
    # location=BQ_REGION
)

# Prepare sample data

You use the [Stack Overflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow) of question and answers hosted on BigQuery.

> This public dataset is hosted in Google BigQuery and is included in BigQuery's 1TB/mo of free tier processing. This means that each user receives 1TB of free BigQuery processing every month, which can be used to run queries on this public dataset.

The BigQuery table is too large to fit into memory, so you need to write a generator called `query_bigquery_chunks` to yield chunks of the dataframe for processing. Additionally, an extra column `title_with_body` is added, which is a concatenation of the question title and body.

## Query dataset

In [24]:
QUERY_TEMPLATE = """
        SELECT DISTINCT q.id, 
           q.title, 
           q.body, 
           q.score, 
           q.tags,
        FROM (
            SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` 
            WHERE score > 0 
            ORDER BY view_count DESC
            ) AS q 
        LIMIT {limit} OFFSET {offset};
        """


def query_bigquery_chunks(
    max_rows: int, 
    rows_per_chunk: int, 
    start_chunk: int = 0
) -> Generator[pd.DataFrame, Any, None]:
    
    for offset in range(start_chunk, max_rows, rows_per_chunk):
        query = QUERY_TEMPLATE.format(limit=rows_per_chunk, offset=offset)
        query_job = bq_client.query(query)
        rows = query_job.result()
        df = rows.to_dataframe()
        df["title_with_body"] = df.title + "\n" + df.body
        df['tags_split_1'] = df['tags'].apply(lambda x: x.split('|', maxsplit=1)[0])
        df['tags_split_2'] = df['tags'].apply(lambda x: x.rsplit('|', maxsplit=1)[-1])
        df.drop(columns=["title","body","tags"], inplace=True)
        yield df

### *(Optional)* sample dataset

In [25]:
# Get a dataframe of 1000 rows for demonstration purposes
df_test = next(
    query_bigquery_chunks(
        max_rows=100, 
        rows_per_chunk=100
    )
)

# Examine the data
print(f"df shape: {df_test.shape}")
df_test.head(3)

df shape: (100, 5)


,id,score,title_with_body,tags_split_1,tags_split_2
0,54839558,1,Call JS onsubmit only when form is valid\n<p>I...,javascript,bootstrap-4
1,54736309,1,How to use output for one function as paramete...,python,parameters
2,54991610,1,How to look up previous values in an R data fr...,r,r


### Converting data to embedding vectors

#### TL;DR

**Embedding Vector format for Vertex AI Vector Search**

> Understand vector format and available filterting methods, e.g., allow and deny lists, numeric filterting, and crowding (i.e., force diversity in retrieved results

**Doc references**
* API reference for [IndexDatapoint](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.types.IndexDatapoint)
* Vector Search [filtering documentation](https://cloud.google.com/vertex-ai/docs/vector-search/filtering)

**embedding_vector**
* Encode the file using UTF-8.
* Make each line a valid `.json` object to be interpreted as a record.
* Include in each record a field named `id` that requires a valid UTF-8 string that is the ID of the vector.
* Include in each record a field named embedding that requires an array of numbers. This is the feature vector.

Filtering with **String Namespaces**

The value of the field `restricts`, if present, should be an array of objects, each is turned into a TokenNamespace in restricts.

For each vector's record, add a field called restricts, to contain an array of objects, each of which is a namespace.

* Each object must have a field named namespace. This field is the TokenNamespace.namespace, namespace.
* The value of the field allow, if present, is an array of strings. This array of strings is the TokenNamespace.string_tokens list.
* The value of the field deny, if present, is an array of strings. This array of strings is the TokenNamespace.string_denylist_tokens list.

```
{
    "id": "42", 
    "embedding": [0.5, 1.0], 
    "restricts": [
        {"namespace": "class","allow": ["cat", "pet"]},
        {"namespace": "category", "allow": ["feline"]}
    ]
}
{
    "id": "43", 
    "embedding": [0.6, 1.0], 
    "restricts": [
        {"namespace": "class", "allow": ["dog", "pet"]},
        {"namespace": "category", "allow": ["canine"]}
    ]
}
```

Filtering with **Numeric namespaces**
For each vector's record, add a field called `numeric_restricts`, to contain an array of objects, each of which is a numeric restrict.

* Each object must have a field named namespace. This field is the NumericRestrictNamespace.namespace, namespace.
* Each object must have one of `value_int`, `value_float`, and `value_double`.
* Each object must not have a field named op. This field is only for query.

```
{
    "id": "42", 
    "embedding": [0.5, 1.0], 
    "numeric_restricts": [
        {"namespace": "size", "value_int": 3},
        {"namespace": "ratio", "value_float": 0.1}
    ]
}
```

**crowding tag**

The value of the field `crowding_tag`, if present, should be a string

```
{
    "id": "43", 
    "embedding": [0.6, 1.0], 
    "numeric_restricts": [
        {"namespace": "ratio", "value_float": 0.1}
    ],
    "crowding_tag": "shoes"
}
```

#### Helper functions

In [32]:
# Define an embedding method that uses the model
def encode_texts_to_embeddings(sentences: List[str]) -> List[Optional[List[float]]]:
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]
    
# Generator function to yield batches of sentences
def generate_batches(
    sentences: List[str], 
    batch_size: int
) -> Generator[List[str], None, None]:
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]
        
def encode_text_to_embedding_batched(
    sentences: List[str], 
    api_calls_per_second: int = 10, 
    batch_size: int = 5
) -> Tuple[List[bool], np.ndarray]:

    embeddings_list: List[List[float]] = []

    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total=math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return is_successful, embeddings_list_successful

def create_emb_vector_files(
    bq_num_rows: int = 1000,
    bq_chunk_size: int = 100,
    bq_num_chunks: int = 10,
    start_chunk: int = 0,
    api_calls_per_sec: int = 50,
    items_per_request: int = 5,
    emb_file_path: str = None
):
    print(f"bq_num_rows       : {bq_num_rows}")
    print(f"bq_chunk_size     : {bq_chunk_size}")
    print(f"bq_num_chunks     : {bq_num_chunks}")
    print(f"start_chunk       : {start_chunk}")
    print(f"api_calls_per_sec : {api_calls_per_sec}")
    print(f"items_per_request : {items_per_request}")
    print(f"emb_file_path     : {emb_file_path}")
    
    rows_list = []
    
    # Loop through each generated dataframe, convert
    for i, df in tqdm(
        enumerate(
            query_bigquery_chunks(
                max_rows=bq_num_rows, 
                rows_per_chunk=bq_chunk_size, 
                start_chunk=start_chunk
            )
        ),
        total=bq_num_chunks,# - start_chunk,
        position=-1,
        desc="Chunk of rows from BigQuery",
    ):
        
        print(f"Starting: {i} of {bq_num_chunks} loops")
        
        # Create a unique output file for each chunk
        chunk_path = emb_file_path.joinpath(
            f"{emb_file_path.stem}_{i+start_chunk}.json"
        )
        with open(chunk_path, "a") as f:
        
            id_chunk = df.id
            scores_chunk = df.score
            tags_1_chunk = df.tags_split_1
            tags_2_chunk = df.tags_split_2

            # Convert batch to embeddings
            is_successful, question_chunk_embeddings = encode_text_to_embedding_batched(
                sentences=df.title_with_body.tolist(), #[:500]
                api_calls_per_second=api_calls_per_sec,
                batch_size=items_per_request,
            )

            embeddings_formatted = [
                json.dumps(
                    {
                        "id": str(id),
                        "embedding": [
                            str(value) for value in embedding
                        ],
                        "tag": str(r_tag),               # restricts_allow
                        "score": int(score),             # numeric_restricts
                        "crowding_tag": str(c_tag)
                    }
                )
                + "\n"
                for id, embedding, r_tag, score, c_tag in zip(
                    id_chunk[is_successful], 
                    question_chunk_embeddings, 
                    tags_1_chunk, 
                    scores_chunk, 
                    tags_2_chunk
                )
            ]
            f.writelines(embeddings_formatted)

            print(f"tags_1_chunk              : {len(tags_1_chunk)}")
            print(f"tags_2_chunk              : {len(tags_2_chunk)}")
            print(f"scores_chunk              : {len(scores_chunk)}")
            print(f"question_chunk_embeddings : {len(question_chunk_embeddings)}")
            print(f"id_chunk[is_successful]   : {len(id_chunk[is_successful])}")

            # Delete the DataFrame and any other large data structures
            del df
            gc.collect()
    
    print("loops complete...\n")
    print(f"len(embeddings_formatted)    : {len(embeddings_formatted)}")
    print(f"len(embeddings_formatted[0]) : {len(embeddings_formatted[0])}")
    
    return embeddings_formatted[0]

#### Test helper functions

In [33]:
# # Encode a subset of questions for validation
# questions = df_test.title.tolist()[:50]

# is_successful, question_embeddings = encode_text_to_embedding_batched(
#     sentences=df.title.tolist()[:50],
# )
# print(question_embeddings.shape)

# DIMENSIONS = len(question_embeddings[0])
# print(DIMENSIONS)

# # Filter for successfully embedded sentences
# questions = np.array(questions)[is_successful]

# print(questions.shape)
# questions[0]

### Create Vector Embedding json

In [65]:
import gc
import json
import tempfile
from pathlib import Path

# Create temporary file to write embeddings to
emb_json_file_path = Path(tempfile.mkdtemp())
print(f"emb_json_file_path: {emb_json_file_path}")

emb_json_file_path: /var/tmp/tmpm4k5k6gq


In [66]:
BQ_NUM_ROWS       = 5000               # position to stop
BQ_CHUNK_SIZE     = 1000                # incrementation
NEXT_START        = 2000                  # position to start

BQ_NUM_CHUNKS     = math.ceil(BQ_NUM_ROWS / BQ_CHUNK_SIZE)
API_CALLS_PER_SEC = 300 / 60

print(f"BQ_NUM_CHUNKS     : {BQ_NUM_CHUNKS}")
print(f"API_CALLS_PER_SEC : {API_CALLS_PER_SEC}")

BQ_NUM_CHUNKS     : 5
API_CALLS_PER_SEC : 5.0


In [67]:
sample_formatted_emb = create_emb_vector_files(
    bq_num_rows = BQ_NUM_ROWS,
    bq_chunk_size = BQ_CHUNK_SIZE,
    bq_num_chunks = BQ_NUM_CHUNKS,
    start_chunk = NEXT_START,
    api_calls_per_sec = API_CALLS_PER_SEC,
    items_per_request = 5,
    emb_file_path = emb_json_file_path
)

bq_num_rows       : 5000
bq_chunk_size     : 1000
bq_num_chunks     : 5
start_chunk       : 2000
api_calls_per_sec : 5.0
items_per_request : 5
emb_file_path     : /var/tmp/tmpm4k5k6gq


Chunk of rows from BigQuery:   0%|          | 0/5 [00:00<?, ?it/s]

Starting: 0 of 5 loops


  0%|          | 0/200 [00:00<?, ?it/s]

tags_1_chunk              : 1000
tags_2_chunk              : 1000
scores_chunk              : 1000
question_chunk_embeddings : 1000
id_chunk[is_successful]   : 1000
Starting: 1 of 5 loops


  0%|          | 0/200 [00:00<?, ?it/s]

tags_1_chunk              : 1000
tags_2_chunk              : 1000
scores_chunk              : 1000
question_chunk_embeddings : 1000
id_chunk[is_successful]   : 1000
Starting: 2 of 5 loops


  0%|          | 0/200 [00:00<?, ?it/s]

tags_1_chunk              : 1000
tags_2_chunk              : 1000
scores_chunk              : 1000
question_chunk_embeddings : 1000
id_chunk[is_successful]   : 1000
loops complete...

len(embeddings_formatted)    : 1000
len(embeddings_formatted[0]) : 18638


In [44]:
# sample_formatted_emb

### Save to Cloud Storage

In [68]:
REMOTE_GCS_FOLDER = f"{BUCKET_URI}/{PREFIX}/embedding_indexes/{emb_json_file_path.stem}/"
print(f"REMOTE_GCS_FOLDER: {REMOTE_GCS_FOLDER}")

REMOTE_GCS_FOLDER: gs://vvs-vectorio-pubv3-hybrid-vertex/vvs-vectorio-pubv3/embedding_indexes/tmpm4k5k6gq/


In [69]:
! gsutil -m cp -r {emb_json_file_path}/* {REMOTE_GCS_FOLDER}

! gsutil ls $REMOTE_GCS_FOLDER

Copying file:///var/tmp/tmpm4k5k6gq/tmpm4k5k6gq_2001.json [Content-Type=application/json]...
Copying file:///var/tmp/tmpm4k5k6gq/tmpm4k5k6gq_2000.json [Content-Type=application/json]...
Copying file:///var/tmp/tmpm4k5k6gq/tmpm4k5k6gq_2002.json [Content-Type=application/json]...
- [3/3 files][ 53.2 MiB/ 53.2 MiB] 100% Done                                    
Operation completed over 3 objects/53.2 MiB.                                     
gs://vvs-vectorio-pubv3-hybrid-vertex/vvs-vectorio-pubv3/embedding_indexes/tmpm4k5k6gq/tmpm4k5k6gq_2000.json
gs://vvs-vectorio-pubv3-hybrid-vertex/vvs-vectorio-pubv3/embedding_indexes/tmpm4k5k6gq/tmpm4k5k6gq_2001.json
gs://vvs-vectorio-pubv3-hybrid-vertex/vvs-vectorio-pubv3/embedding_indexes/tmpm4k5k6gq/tmpm4k5k6gq_2002.json


### Write to parquet

In [70]:
# Create temporary file to write embeddings to
parquet_file_path = Path(tempfile.mkdtemp())
print(f"parquet_file_path: {parquet_file_path}")

EMB_PARQ_GCS_DIR = f"{BUCKET_URI}/emb_vector_parquet"
SO_PARQUET_GCS_DIR = f"{EMB_PARQ_GCS_DIR}/so_{NEXT_START}_{BQ_NUM_ROWS}_{BQ_CHUNK_SIZE}/{parquet_file_path.stem}/"
print(f"SO_PARQUET_GCS_DIR: {SO_PARQUET_GCS_DIR}")

parquet_file_path: /var/tmp/tmpsgm4txp8
SO_PARQUET_GCS_DIR: gs://vvs-vectorio-pubv3-hybrid-vertex/emb_vector_parquet/so_2000_5000_1000/tmpsgm4txp8/


In [71]:
PARQUET_GCS_FILE_LIST = []

for f in emb_json_file_path.iterdir():
    
    local_f = os.path.abspath(str(f))
    dest_file = os.path.join(SO_PARQUET_GCS_DIR, f"{f.stem}.parquet")
    
    # print(f"reading from: {local_f}")
    table = pj.read_json(local_f)
    
    # print(f"saving to: {dest_file}")
    pq.write_table(table, dest_file)
    
    PARQUET_GCS_FILE_LIST.append(dest_file)
    
print(f"saved parquet files to: {SO_PARQUET_GCS_DIR}\n")
PARQUET_GCS_FILE_LIST

saved parquet files to: gs://vvs-vectorio-pubv3-hybrid-vertex/emb_vector_parquet/so_2000_5000_1000/tmpsgm4txp8/



['gs://vvs-vectorio-pubv3-hybrid-vertex/emb_vector_parquet/so_2000_5000_1000/tmpsgm4txp8/tmpm4k5k6gq_2001.parquet',
 'gs://vvs-vectorio-pubv3-hybrid-vertex/emb_vector_parquet/so_2000_5000_1000/tmpsgm4txp8/tmpm4k5k6gq_2000.parquet',
 'gs://vvs-vectorio-pubv3-hybrid-vertex/emb_vector_parquet/so_2000_5000_1000/tmpsgm4txp8/tmpm4k5k6gq_2002.parquet']

In [72]:
# validate parquet files
df_from_pq = pd.read_parquet(PARQUET_GCS_FILE_LIST[0])

print(df_from_pq.shape)
df_from_pq.head(5)

(1000, 5)


,id,embedding,tag,score,crowding_tag
0,43154170,"[-0.024622129276394844, -0.005234652664512396,...",security,16,cors
1,43441856,"[0.01128651574254036, -0.0018839503172785044, ...",javascript,387,ecmascript-6
2,43460880,"[-0.04847663268446922, -0.01450541615486145, 0...",unit-testing,20,xunit
3,43373082,"[-0.002033930504694581, -0.05231621116399765, ...",java,11,vaadin7
4,47416861,"[0.006317105144262314, -0.05412781983613968, 0...",python,23,keras-layer


### Write test sets locally

In [73]:
LOCAL_TEST_DIR = f"data/stack_overflow_parquet_{VERSION}"
LOCAL_TEST_DATA_DIR = f"{LOCAL_TEST_DIR}/files"

In [74]:
os.makedirs(LOCAL_TEST_DATA_DIR, exist_ok=True)

In [75]:
LOCAL_PARQUEST_FILE_LIST = []

for file in PARQUET_GCS_FILE_LIST:

    file_name = file.rsplit('/', maxsplit=1)[-1]
    print(file_name)
    
    LOCAL_PARQUET_FILE = f"{LOCAL_TEST_DATA_DIR}/so_{file_name}"
    
    df_tmp = pd.read_parquet(file)
    df_tmp.to_parquet(LOCAL_PARQUET_FILE)
    
    LOCAL_PARQUEST_FILE_LIST.append(LOCAL_PARQUET_FILE)
    
    # Delete the DataFrame and any other large data structures
    del df_tmp
    gc.collect()

tmpm4k5k6gq_2001.parquet
tmpm4k5k6gq_2000.parquet
tmpm4k5k6gq_2002.parquet


In [76]:
LOCAL_PARQUEST_FILE_LIST

['data/stack_overflow_parquet_pubv3/files/so_tmpm4k5k6gq_2001.parquet',
 'data/stack_overflow_parquet_pubv3/files/so_tmpm4k5k6gq_2000.parquet',
 'data/stack_overflow_parquet_pubv3/files/so_tmpm4k5k6gq_2002.parquet']

# Create Vector Search Index, and deploy to Index Endpoint

> This will create an index with a single vector; use this to test the import and export classes

Steps:
1. Create single dummy embedding vector to create Vector Search index
2. Create index endpoint
3. deploy index to index endpoint

### TL;DR

**VS Index Config**
* Set index's update method for *streaming updates*
* See [Create and manage your index](https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index) documentation to better understand `Shard sizes` and the `machine types` available
* See [Tuning the index](https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index#tuning_the_index) documentation to learn how config parameters impact recall and latency

**VS Index Endpoint Config**
* Use public endpoints unless VPC is a networking requirement

**VS Deployed Index ID**: `deployed_index_id` (str) 
* Required. The user specified ID of the `DeployedIndex`
* can be up to 128 characters long, 
* must start with a letter and only contain letters, numbers, and underscores.
* The ID must be unique within the project it is created in

`VPC_NETWORK_NAME`
* if index will be **deployed to a private endpoint** within a VPC network, edit this with the name of your VPC network name
* if index will be **deployed to a public endpoint**, leave blank

> For details on configuring VPC for Vertex AI Vector Search, see **[Set up a VPC Network Peering connection](https://cloud.google.com/vertex-ai/docs/vector-search/setup/vpc)**

<div class="alert alert-block alert-warning">
    <b>⚠️ if deploying index to index endpoint within a VPC network, you must interact with the index endpoint from within the VPC network, i.e., run this notebook in a Vertex Workbench instance within that VPC ⚠️</b>
</div>

## Index and Endpoint config

In [59]:
from google.protobuf import struct_pb2

*To list current indexes*

```
!gcloud ai indexes list --project=$PROJECT_ID --region=$REGION
```

In [48]:
CREATE_NEW_VS_INDEX = False

# if using exsiting index
if not CREATE_NEW_VS_INDEX:
    EXISTING_INDEX_ID = "1081325705452584960" # TODO
    EXISTING_INDEX_NAME = f'projects/{PROJECT_NUM}/locations/{REGION}/indexes/{EXISTING_INDEX_ID}'
    print(f"EXISTING_INDEX_NAME  : {EXISTING_INDEX_NAME}")

EXISTING_INDEX_NAME  : projects/934903580331/locations/us-central1/indexes/1081325705452584960


In [49]:
# specify VPC network or leave blank
VPC_NETWORK_NAME             = ""  # e.g., "your-vpc-name" | ""

if VPC_NETWORK_NAME:
    USE_PUBLIC_ENDPOINTS     = False
    # full VPC network name
    VPC_NETWORK_FULL         = f"projects/{PROJECT_NUM}/global/networks/{VPC_NETWORK_NAME}"
    print(f"VPC_NETWORK_NAME : {VPC_NETWORK_NAME}")
    print(f"VPC_NETWORK_FULL : {VPC_NETWORK_FULL}")
else:
    USE_PUBLIC_ENDPOINTS     = True
    VPC_NETWORK_FULL         = None

print(f"USE_PUBLIC_ENDPOINTS = {USE_PUBLIC_ENDPOINTS}")

USE_PUBLIC_ENDPOINTS = True


In [80]:
# =========================================================
# ANN index config
# =========================================================
DISPLAY_NAME        = f"soverflow_{PREFIX}".replace("-","_")
DESCRIPTION         = "sample index for vectorio demo"
APPROX_NEIGHBORS    = 150
DISTANCE_MEASURE    = "DOT_PRODUCT_DISTANCE"
LEAF_NODE_EMB_COUNT = 500
LEAF_SEARCH_PERCENT = 80
DIMENSIONS          = 768
INDEX_UPDATE_METHOD = aipv1.Index.IndexUpdateMethod.STREAM_UPDATE # "STREAM_UPDATE"
INDEX_SHARD_SIZE    = "SHARD_SIZE_MEDIUM"

# =========================================================
# index endpoint config
# =========================================================
ENDPOINT_DISPLAY_NAME = f'{DISPLAY_NAME}_endpoint'
ENDPOINT_DESCRIPTION  = "index endpoint for vectorio demo"
print(f"ENDPOINT_DISPLAY_NAME : {ENDPOINT_DISPLAY_NAME}")
print(f"ENDPOINT_DESCRIPTION  : {ENDPOINT_DESCRIPTION}")
print(f"USE_PUBLIC_ENDPOINTS  : {USE_PUBLIC_ENDPOINTS}")

# =========================================================
# Deployed index
# =========================================================
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
DEPLOYED_INDEX_ID = f"{DISPLAY_NAME.replace('-', '_')}_{timestamp}"
MACHINE_TYPE      = "e2-standard-16"
MIN_REPLICAS      = 1
MAX_REPLICAS      = 1
print(f"DEPLOYED_INDEX_ID     : {DEPLOYED_INDEX_ID}")
print(f"# characters (< 128)  : {len(DEPLOYED_INDEX_ID)}")
print(f"MACHINE_TYPE          : {MACHINE_TYPE}")
print(f"MIN_REPLICAS          : {MIN_REPLICAS}")
print(f"MAX_REPLICAS          : {MAX_REPLICAS}")

# =========================================================
# set index client
# =========================================================
project_config = {
    "region": REGION,
    "project_id": PROJECT_ID,
}
endpoint = "{}-aiplatform.googleapis.com".format(project_config["region"])
index_client = aipv1.IndexServiceClient(
    client_options=dict(api_endpoint=endpoint)
)

ENDPOINT_DISPLAY_NAME : soverflow_vvs_vectorio_pubv3_endpoint
ENDPOINT_DESCRIPTION  : index endpoint for vectorio demo
USE_PUBLIC_ENDPOINTS  : True
DEPLOYED_INDEX_ID     : soverflow_vvs_vectorio_pubv3_20240130131739
# characters (< 128)  : 43
MACHINE_TYPE          : e2-standard-16
MIN_REPLICAS          : 1
MAX_REPLICAS          : 1


## Create dummy data for index

In [51]:
# =========================================================
# initial emb vector
# =========================================================
LOCAL_INIT_FILE = "embeddings_0.json"
CONTENTS_EMB_DIR = "init_index"

EMBEDDING_BLOB_NAME = f"{CONTENTS_EMB_DIR}/{LOCAL_INIT_FILE}"
CONTENTS_URI = f"{BUCKET_URI}/{CONTENTS_EMB_DIR}"

print(f"CONTENTS_URI           : {CONTENTS_URI}")
print(f"EMBEDDING_BLOB_NAME    : {EMBEDDING_BLOB_NAME}")

CONTENTS_URI           : gs://vvs-vectorio-pubv3-hybrid-vertex/init_index
EMBEDDING_BLOB_NAME    : init_index/embeddings_0.json


In [52]:
if CREATE_NEW_VS_INDEX:
    
    # dummy embedding
    init_embedding = {
        "id": str(uuid.uuid4()),
        "embedding": list(np.zeros(DIMENSIONS))
    }

    # dump embedding to a local file
    with open(LOCAL_INIT_FILE, "w") as f:
        json.dump(init_embedding, f)

    # write embedding to Cloud Storage
    ! gsutil cp $LOCAL_INIT_FILE $CONTENTS_URI/$LOCAL_INIT_FILE

In [53]:
!gsutil ls $CONTENTS_URI

gs://vvs-vectorio-pubv3-hybrid-vertex/init_index/embeddings_0.json


## Define Index Configuration Spec

In [54]:
VS_CONFIG_SPEC = {
    "index_display_name": DISPLAY_NAME,
    "contents_delta_uri": CONTENTS_URI,
    "dimensions": DIMENSIONS,
    "approximate_neighbors_count": APPROX_NEIGHBORS,
    "distance_measure_type": DISTANCE_MEASURE,
    "leaf_node_embedding_count": LEAF_NODE_EMB_COUNT,
    "leaf_nodes_to_search_percent": LEAF_SEARCH_PERCENT,
    "index_shard_size": INDEX_SHARD_SIZE,
    "index_update_method": INDEX_UPDATE_METHOD,
    "description": DESCRIPTION,
    "labels": {
        "prefix": PREFIX
    },
    "index_endpoint_display_name": ENDPOINT_DISPLAY_NAME,
    "index_endpoint_description": ENDPOINT_DESCRIPTION,
    "deployed_index_id": DEPLOYED_INDEX_ID,
}
VS_CONFIG_SPEC

{'index_display_name': 'soverflow_vvs_vectorio_pubv3',
 'contents_delta_uri': 'gs://vvs-vectorio-pubv3-hybrid-vertex/init_index',
 'dimensions': 768,
 'approximate_neighbors_count': 150,
 'distance_measure_type': 'DOT_PRODUCT_DISTANCE',
 'leaf_node_embedding_count': 500,
 'leaf_nodes_to_search_percent': 80,
 'index_shard_size': 'SHARD_SIZE_MEDIUM',
 'index_update_method': <IndexUpdateMethod.STREAM_UPDATE: 2>,
 'description': 'sample index for vectorio demo',
 'labels': {'prefix': 'vvs-vectorio-pubv3'},
 'index_endpoint_display_name': 'soverflow_vvs_vectorio_pubv3_endpoint',
 'index_endpoint_description': 'index endpoint for vectorio demo',
 'deployed_index_id': 'soverflow_vvs_vectorio_pubv3_20240130192127'}

In [82]:
TREE_AH_CONFIG = struct_pb2.Struct(
    fields={
        "leafNodeEmbeddingCount": struct_pb2.Value(
            number_value=VS_CONFIG_SPEC["leaf_node_embedding_count"]
        ),
        "leafNodesToSearchPercent": struct_pb2.Value(
            number_value=VS_CONFIG_SPEC[
                "leaf_nodes_to_search_percent"
            ]
        ),
    }
)
ALGORITHM_CONFIG = struct_pb2.Struct(
    fields={"treeAhConfig": struct_pb2.Value(struct_value=TREE_AH_CONFIG)}
)

In [85]:
# ALGORITHM_CONFIG

In [86]:
VS_CONFIG = struct_pb2.Struct(
    fields={
        "dimensions": struct_pb2.Value(
            number_value=VS_CONFIG_SPEC["dimensions"]
        ),
        "approximateNeighborsCount": struct_pb2.Value(
            number_value=VS_CONFIG_SPEC["approximate_neighbors_count"]
        ),
        "distanceMeasureType": struct_pb2.Value(
            string_value=VS_CONFIG_SPEC["distance_measure_type"]
        ),
        "algorithmConfig": struct_pb2.Value(struct_value=ALGORITHM_CONFIG),
        "shardSize": struct_pb2.Value(string_value=VS_CONFIG_SPEC["index_shard_size"]),
    }
)
VS_METADATA = struct_pb2.Struct(
    fields={
        "config": struct_pb2.Value(struct_value=VS_CONFIG),
        "contentsDeltaUri": struct_pb2.Value(
            string_value=VS_CONFIG_SPEC["contents_delta_uri"]
        ),
    }
)

In [88]:
# VS_METADATA

In [89]:
INDEX_REQUEST = {
    "display_name": VS_CONFIG_SPEC["index_display_name"],
    "description": VS_CONFIG_SPEC["description"],
    "metadata": struct_pb2.Value(struct_value=VS_METADATA),
    "index_update_method": VS_CONFIG_SPEC["index_update_method"],
}
  
PARENT = f"projects/{project_config['project_id']}/locations/{project_config['region']}"

In [91]:
# INDEX_REQUEST

## Create index 

> see **Troubleshooting** section below if encountering errors

In [55]:
if CREATE_NEW_VS_INDEX:
    
    print(f"Creating new index: {VS_CONFIG_SPEC['index_display_name']} ...")
    start = time.time()
    create_lro = index_client.create_index(parent=PARENT, index=INDEX_REQUEST)
    
    # Poll the operation until it's done successfullly.
    while True:
        if create_lro.done():
            break
        time.sleep(5)
        
    index = create_lro.result()
    my_vs_index = aiplatform.MatchingEngineIndex(index.name)
    
    end = time.time()
    print(f"elapsed time: {round((end - start), 2)}")
    
else:
    my_vs_index = aiplatform.MatchingEngineIndex(EXISTING_INDEX_NAME)
    
INDEX_RESOURCE_NAME = my_vs_index.resource_name
INDEX_DISPLAY_NAME  = my_vs_index.display_name

print(f"INDEX_RESOURCE_NAME : {INDEX_RESOURCE_NAME}")
print(f"INDEX_DISPLAY_NAME  : {INDEX_DISPLAY_NAME}")

INDEX_RESOURCE_NAME : projects/934903580331/locations/us-central1/indexes/1081325705452584960
INDEX_DISPLAY_NAME  : soverflow_vvs_vectorio_pubv3


In [56]:
# get all index config to dictionary 
my_vs_index.to_dict()

{'name': 'projects/934903580331/locations/us-central1/indexes/1081325705452584960',
 'displayName': 'soverflow_vvs_vectorio_pubv3',
 'description': 'sample index for vectorio demo',
 'metadataSchemaUri': 'gs://google-cloud-aiplatform/schema/matchingengine/metadata/nearest_neighbor_search_1.0.0.yaml',
 'metadata': {'config': {'dimensions': 768.0,
   'approximateNeighborsCount': 150.0,
   'distanceMeasureType': 'DOT_PRODUCT_DISTANCE',
   'algorithmConfig': {'treeAhConfig': {'leafNodeEmbeddingCount': '500',
     'leafNodesToSearchPercent': 80.0}},
   'shardSize': 'SHARD_SIZE_MEDIUM'}},
 'deployedIndexes': [{'indexEndpoint': 'projects/934903580331/locations/us-central1/indexEndpoints/5739455095037231104',
   'deployedIndexId': 'soverflow_vvs_vectorio_pubv3_20240130131739'}],
 'etag': 'AMEw9yMreqkZWIbaAt6IqdpJ2c_z8XOT30Cwd7HFJHIFPz1FpFGVFgv_0PLQUNHKYfs=',
 'createTime': '2024-01-30T13:26:08.725251Z',
 'updateTime': '2024-01-30T13:26:17.176312Z',
 'indexStats': {'vectorsCount': '1', 'shardsC

## Deploy index to endpoint

*To list current index endpoints:*
```
!gcloud ai index-endpoints list --project=$PROJECT_ID --region=$REGION
```

### Create index endpoint

In [57]:
CREATE_NEW_VS_INDEX_ENDPOINT = False

# if using exsiting index endpoint
if not CREATE_NEW_VS_INDEX_ENDPOINT:
    EXISTING_ENDPOINT_ID = "5739455095037231104" # TODO
    EXISTING_ENDPOINT_NAME = f'projects/{PROJECT_NUM}/locations/{REGION}/indexEndpoints/{EXISTING_ENDPOINT_ID}'
    print(f"EXISTING_ENDPOINT_NAME  : {EXISTING_ENDPOINT_NAME}")

EXISTING_ENDPOINT_NAME  : projects/934903580331/locations/us-central1/indexEndpoints/5739455095037231104


In [58]:
if CREATE_NEW_VS_INDEX_ENDPOINT:
    
    print(f"Creating new index endpoint: {VS_CONFIG_SPEC['index_endpoint_display_name']} ...")
    start = time.time()
    my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
        display_name=VS_CONFIG_SPEC["index_endpoint_display_name"],
        description=VS_CONFIG_SPEC["index_endpoint_description"],
        network=VPC_NETWORK_FULL if not USE_PUBLIC_ENDPOINTS else None,
        public_endpoint_enabled=USE_PUBLIC_ENDPOINTS,
        sync=True,
    )
    end = time.time()
    print(f"elapsed time: {round((end - start), 2)}")
    
else:
    my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(EXISTING_ENDPOINT_NAME)
    
ENDPOINT_DISPLAY_NAME  = my_index_endpoint.display_name
ENDPOINT_RESOURCE_NAME = my_index_endpoint.resource_name

print(f"ENDPOINT_DISPLAY_NAME  : {ENDPOINT_DISPLAY_NAME}")
print(f"ENDPOINT_RESOURCE_NAME : {ENDPOINT_RESOURCE_NAME}")

ENDPOINT_DISPLAY_NAME  : soverflow_vvs_vectorio_pubv3_endpoint
ENDPOINT_RESOURCE_NAME : projects/934903580331/locations/us-central1/indexEndpoints/5739455095037231104


### Deploy to index endpoint

In [59]:
DEPLOY_NEW_VS_INDEX = False

# # if using exsiting deployed index
# if not DEPLOY_NEW_VS_INDEX:
#     EXISTING_DEPLOYED_INDEX_ID = "..." # TODO
#     EXISTING_DEPLOYED_INDEX_NAME = f'projects/{PROJECT_NUM}/locations/{REGION}/indexEndpoints/{EXISTING_DEPLOYED_INDEX_ID}'
#     print(f"EXISTING_DEPLOYED_INDEX_NAME  : {EXISTING_DEPLOYED_INDEX_NAME}")

In [60]:
if DEPLOY_NEW_VS_INDEX:
    
    print(f"Deploying index to endpoint: {ENDPOINT_DISPLAY_NAME} ...")
    start = time.time()

    deployed_index = my_index_endpoint.deploy_index(
        index=my_vs_index,
        deployed_index_id=VS_CONFIG_SPEC['deployed_index_id'],
        min_replica_count=MIN_REPLICAS,
        max_replica_count=MAX_REPLICAS,
    )
    
    end = time.time()
    print(f"elapsed time: {round((end - start), 2)}")
else:
    deployed_index = aiplatform.MatchingEngineIndexEndpoint(EXISTING_ENDPOINT_NAME)
    
PUBLIC_ENDPOINT_URL = deployed_index.public_endpoint_domain_name
DEPLOYED_INDEX_ID_TEST = deployed_index.deployed_indexes[0].id

print(f"PUBLIC_ENDPOINT_URL    : {PUBLIC_ENDPOINT_URL}")
print(f"DEPLOYED_INDEX_ID_TEST : {DEPLOYED_INDEX_ID_TEST}")

PUBLIC_ENDPOINT_URL    : 139422526.us-central1-934903580331.vdb.vertexai.goog
DEPLOYED_INDEX_ID_TEST : soverflow_vvs_vectorio_pubv3_20240130131739


In [61]:
print(f"Deployed indexes on the index endpoint:")
for d in my_index_endpoint.deployed_indexes:
    print(f"    {d.id}")

Deployed indexes on the index endpoint:
    soverflow_vvs_vectorio_pubv3_20240130131739


### Get Index and Endpoint IDs

In [62]:
MY_INDEX_ID          = INDEX_RESOURCE_NAME.split("/")[5]
MY_INDEX_ENDPOINT_ID = ENDPOINT_RESOURCE_NAME.split("/")[5]

print(f"MY_INDEX_ID          = {MY_INDEX_ID}")
print(f"MY_INDEX_ENDPOINT_ID = {MY_INDEX_ENDPOINT_ID}")

MY_INDEX_ID          = 1081325705452584960
MY_INDEX_ENDPOINT_ID = 5739455095037231104


## Check vector count

In [64]:
number_of_vectors = sum(
    aiplatform.MatchingEngineIndex(
        deployed_index.index
    )._gca_resource.index_stats.vectors_count
    for deployed_index in my_index_endpoint.deployed_indexes
)

print(f"Actual: {number_of_vectors}")

Actual: 1


### Save notebook config 

> to easily use in other GCP related notebooks

**TODO**
* add more variables to be re-used in import class test
* add more variables to be re-used in export class test

In [77]:
LOCAL_PARQUEST_FILE_STR = '|'.join(LOCAL_PARQUEST_FILE_LIST)
LOCAL_PARQUEST_FILE_STR

'data/stack_overflow_parquet_pubv3/files/so_tmpm4k5k6gq_2001.parquet|data/stack_overflow_parquet_pubv3/files/so_tmpm4k5k6gq_2000.parquet|data/stack_overflow_parquet_pubv3/files/so_tmpm4k5k6gq_2002.parquet'

In [81]:
config = f"""
PREFIX                   = \"{PREFIX}\"
VERSION                  = \"{VERSION}\"

PROJECT_ID               = \"{PROJECT_ID}\"
PROJECT_NUM              = \"{PROJECT_NUM}\"

REGION                   = \"{REGION}\"
BQ_REGION                = \"{BQ_REGION}\"

VERTEX_SA                = \"{VERTEX_SA}\"

VPC_NETWORK_NAME         = \"{VPC_NETWORK_NAME}\"
VPC_NETWORK_FULL         = \"{VPC_NETWORK_FULL}\"

USE_PUBLIC_ENDPOINTS     = \"{USE_PUBLIC_ENDPOINTS}\"

BUCKET_NAME              = \"{BUCKET_NAME}\"
BUCKET_URI               = \"{BUCKET_URI}\"

REMOTE_GCS_FOLDER        = \"{REMOTE_GCS_FOLDER}\"

SO_PARQUET_GCS_DIR       = \"{SO_PARQUET_GCS_DIR}\"

LOCAL_TEST_DIR           = \"{LOCAL_TEST_DIR}\"
LOCAL_TEST_DATA_DIR      = \"{LOCAL_TEST_DATA_DIR}\"

DIMENSIONS               = \"{DIMENSIONS}\"

INDEX_DISPLAY_NAME       = \"{INDEX_DISPLAY_NAME}\"
INDEX_RESOURCE_NAME      = \"{INDEX_RESOURCE_NAME}\"
MY_INDEX_ID              = \"{MY_INDEX_ID}\"

ENDPOINT_DISPLAY_NAME    = \"{ENDPOINT_DISPLAY_NAME}\"
ENDPOINT_RESOURCE_NAME   = \"{ENDPOINT_RESOURCE_NAME}\"
MY_INDEX_ENDPOINT_ID     = \"{MY_INDEX_ENDPOINT_ID}\"

DEPLOYED_INDEX_ID        = \"{DEPLOYED_INDEX_ID}\"

PUBLIC_ENDPOINT_URL      = \"{PUBLIC_ENDPOINT_URL}\"

LOCAL_PARQUEST_FILE_LIST = \"{LOCAL_PARQUEST_FILE_STR}\"
"""
print(config)


PREFIX                   = "vvs-vectorio-pubv3"
VERSION                  = "pubv3"

PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"

REGION                   = "us-central1"
BQ_REGION                = "US"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

VPC_NETWORK_NAME         = ""
VPC_NETWORK_FULL         = "None"

USE_PUBLIC_ENDPOINTS     = "True"

BUCKET_NAME              = "vvs-vectorio-pubv3-hybrid-vertex"
BUCKET_URI               = "gs://vvs-vectorio-pubv3-hybrid-vertex"

REMOTE_GCS_FOLDER        = "gs://vvs-vectorio-pubv3-hybrid-vertex/vvs-vectorio-pubv3/embedding_indexes/tmpm4k5k6gq/"

SO_PARQUET_GCS_DIR       = "gs://vvs-vectorio-pubv3-hybrid-vertex/emb_vector_parquet/so_2000_5000_1000/tmpsgm4txp8/"

LOCAL_TEST_DIR           = "data/stack_overflow_parquet_pubv3"
LOCAL_TEST_DATA_DIR      = "data/stack_overflow_parquet_pubv3/files"

DIMENSIONS               = "768"

INDEX_DISPLAY_NAME       = "soverflow_vvs

In [82]:
!echo '{config}' | gsutil cp - {BUCKET_URI}/config/notebook_env.py

Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              


# Troubleshooting

If index creation fails, grab `OPERATION_ID` and `FAILED_INDEX_ID` from the operation resource name in the error message, for example:

> `Please check the details in the metadata of operation: projects/934903580331/locations/us-central1/indexes/FAILED_INDEX_ID/operations/OPERATION_ID.`

Then, use `gcloud ai operations describe` to get the error details:

```
!gcloud ai operations describe $OPERATION_ID \
    --index=$FAILED_INDEX_ID \
    --project=$PROJECT_ID \
    --region=$REGION
```

In [ ]:
# OPERATION_ID    = "2710742495469240320"
# FAILED_INDEX_ID = "4846053518957608960"

# !gcloud ai operations describe $OPERATION_ID \
#     --index=$FAILED_INDEX_ID \
#     --project=$PROJECT_ID \
#     --region=$REGION

# Clean-up

> TODO

In [ ]:
# import os

# delete_bucket = False

# # Force undeployment of indexes and delete endpoint
# my_index_endpoint.delete(force=True)

# # Delete indexes
# tree_ah_index.delete()

# if delete_bucket or os.getenv("IS_TESTING"):
#     ! gsutil rm -rf {BUCKET_URI}